In [1]:
!pip install mistralai python-dotenv flask streamlit requests

INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/2.0 MB 1.9 MB/s eta 0:00:01
   -------------------- ------------------- 1.0/2.0 MB 1.9 MB/s eta 0:00:01
   -------------------------- ------------- 1.3/2.0 MB 1.8 MB/s eta 0:00:01
   ------------------------------------ --- 1.8/2.0 MB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 2.0 MB/s  0:00:01

   ----------------------------------------  0/18 [werkzeug]
   ----------------------------------------  0/18 [werkzeug]
   ------ ---------------------------------  3/18 [pydantic-core]
   -------- -------------------------------  4/18 [opentelemetry-proto]
   ------------- --------------------------  6/18 [invoke]
   --------

In [4]:
from dotenv import load_dotenv
import os

load_dotenv()  # reads .env
api_key = os.getenv("MISTRAL_API_KEY")
print("Does the key exist check ?", api_key is not None)

Does the key exist check ? True


In [8]:
from mistralai import Mistral

def mistral_chat(user_message, model="mistral-small-latest", is_json=False, temperature=0.2):
    client = Mistral(api_key=api_key)

    messages = [
        {"role": "user", "content": user_message}
    ]

    kwargs = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
    }

    if is_json:
        kwargs["response_format"] = {"type": "json_object"}

    resp = client.chat.complete(**kwargs)
    return resp.choices[0].message.content

print(mistral_chat("hello, what can you do?"))

Hello! I can help with a wide range of tasks, including:

- **Answering Questions**: General knowledge, science, history, technology, and more.
- **Writing Assistance**: Essays, emails, stories, reports, and creative writing.
- **Coding Help**: Debugging, explaining code, or generating snippets in Python, JavaScript, Java, and other languages.
- **Learning & Education**: Explaining concepts, summarizing topics, or providing study guides.
- **Productivity & Organization**: Task management, note-taking, and planning.
- **Entertainment**: Jokes, trivia, word games, and fun facts.
- **Language Support**: Translation, grammar checks, and language learning tips.
- **Recommendations**: Books, movies, music, recipes, and travel ideas.

Just let me know what you need, and I’ll do my best to assist! 😊


In [15]:
classification_prompt = """
You are a bank customer service bot.

Your task is to assess customer intent and categorize the customer inquiry
after <<<>>> into one of the following predefined categories:

card arrival
change pin
exchange rate
country support
cancel transfer
charge dispute

If the text doesn't fit into any of the above categories,
classify it as:
customer service

You will only respond with the predefined category.
Do not provide explanations or notes.

###
Here are some examples:

Inquiry: How do I know if I will get my card, or if it is lost?
Category: card arrival

Inquiry: I am planning an international trip and would like to inquire about the current exchange rates.
Category: exchange rate

Inquiry: What countries are getting support? I will be traveling and living abroad.
Category: country support

Inquiry: Can I get help starting my computer? I am having difficulty starting it.
Category: customer service

###
<<<
Inquiry: {inquiry}
>>>
Category:
"""

In [20]:
# fixed_prompt = mistral_chat(
#     "Please correct the spelling and grammar of this prompt and return the same prompt with fixes:\n"
#     + classification_prompt
# )
# print(fixed_prompt)

rewrite_instruction = """
You are a prompt editor.

Task: Fix spelling/grammar ONLY. Do NOT change meaning.
Return ONLY the corrected prompt text.
Do NOT add introductions, bullet points, markdown, or explanations.
Do NOT include “Here is…”, “Key corrections…”, or any extra text.
Output must start with the first character of the prompt and end with the last character of the prompt.

PROMPT TO FIX:
<<<
{prompt}
>>>
""".strip()

fixed_prompt = mistral_chat(rewrite_instruction.format(prompt=classification_prompt), temperature=0.0)
print(fixed_prompt)

You are a bank customer service bot.

Your task is to assess customer intent and categorize the customer inquiry
after <<<>>> into one of the following predefined categories:

card arrival
change pin
exchange rate
country support
cancel transfer
charge dispute

If the text doesn't fit into any of the above categories,
classify it as:
customer service

You will only respond with the predefined category.
Do not provide explanations or notes.

###
Here are some examples:

Inquiry: How do I know if I will get my card, or if it is lost?
Category: card arrival

Inquiry: I am planning an international trip and would like to inquire about the current exchange rates.
Category: exchange rate

Inquiry: What countries are getting support? I will be traveling and living abroad.
Category: country support

Inquiry: Can I get help starting my computer? I am having difficulty starting it.
Category: customer service

###
<<<
Inquiry: {inquiry}
>>>
Category:


In [21]:
inquiry_text = "I am inquiring about the availability of your cards in the EU"
category = mistral_chat(fixed_prompt.format(inquiry=inquiry_text))
print("Predicted category:", category)

Predicted category: country support


In [23]:
medical_notes = """
A 60-year-old male patient, Mr. Johnson, presented with symptoms
of increased thirst, frequent urination, fatigue, and unexplained
weight loss. Upon evaluation, he was diagnosed with diabetes,
confirmed by elevated blood sugar levels. Mr. Johnson's weight
is 210 lbs. He has been prescribed Metformin to be taken twice daily
with meals. It was noted during the consultation that the patient is
a current smoker.
"""

In [24]:
extract_prompt = f"""
Extract information from the following medical notes:

{medical_notes}

Return a JSON object that matches this schema exactly:
{{
  "age": {{"type": "integer"}},
  "gender": {{"type": "string", "enum": ["male", "female", "other"]}},
  "diagnosis": {{"type": "string", "enum": ["migraine", "diabetes", "arthritis", "acne"]}},
  "weight": {{"type": "integer"}},
  "smoking": {{"type": "string", "enum": ["yes", "no"]}}
}}
"""

In [25]:
response_json = mistral_chat(extract_prompt, is_json=True)
print(response_json)


{
  "age": 60,
  "gender": "male",
  "diagnosis": "diabetes",
  "weight": 210,
  "smoking": "yes"
}


In [30]:
email = """
Dear mortgage lender,
What's your 30-year fixed-rate APR, how is it compared to the 15-year
fixed rate?
Regards,
Anna
"""

In [39]:
personalization_prompt = f"""
You are a mortgage lender customer service bot. Your task is to create
personalized email responses to address customer questions.

Answer the customer's inquiry using the provided facts below.
Ensure your response is clear, concise, and directly addresses the question.
Address the customer in a friendly and professional manner.
The LAST line of your response MUST be exactly:
Lender Customer Support

Do not omit it.

Return ONLY the email body.
Do NOT include a subject line.
Do NOT use markdown formatting.
Do NOT use bold text.
Output plain text only.

# Facts
0-year fixed-rate: interest rate 6.403%, APR 6.484%
20-year fixed-rate: interest rate 6.329%, APR 6.429%
15-year fixed-rate: interest rate 5.705%, APR 5.848%
10-year fixed-rate: interest rate 5.500%, APR 5.720%
7-year ARM: interest rate 7.011%, APR 7.660%
5-year ARM: interest rate 6.880%, APR 7.754%
3-year ARM: interest rate 6.125%, APR 7.204%
30-year fixed-rate FHA: interest rate 5.527%, APR 6.316%
30-year fixed-rate VA: interest rate 5.684%, APR 6.062%

# Email
{email}
"""

In [40]:
reply = mistral_chat(personalization_prompt)
print(reply)

Dear Anna,

Thank you for reaching out! Our current 30-year fixed-rate APR is 6.484%. In comparison, the 15-year fixed-rate APR is 5.848%. As you can see, the 15-year fixed-rate option has a lower APR, which typically means lower overall interest costs over the life of the loan, though the monthly payments may be higher due to the shorter term.

Please let us know if you have any other questions or need further assistance.

Lender Customer Support


In [35]:
newsletter = """
European AI champion Mistral AI unveiled new large language models and formed an alliance with Microsoft. 

What’s new: Mistral AI introduced two closed models, Mistral Large and Mistral Small (joining Mistral Medium, which debuted quietly late last year). Microsoft invested $16.3 million in the French startup, and it agreed to distribute Mistral Large on its Azure platform and let Mistral AI use Azure computing infrastructure. Mistral AI makes the new models available to try for free here and to use on its La Plateforme and via custom deployments.

Model specs: The new models’ parameter counts, architectures, and training methods are undisclosed. Like the earlier, open source Mistral 7B and Mixtral 8x7B, they can process 32,000 tokens of input context. 

Mistral Large achieved 81.2 percent on the MMLU benchmark, outperforming Anthropic’s Claude 2, Google’s Gemini Pro, and Meta’s Llama 2 70B, though falling short of GPT-4. Mistral Small, which is optimized for latency and cost, achieved 72.2 percent on MMLU.
Both models are fluent in French, German, Spanish, and Italian. They’re trained for function calling and JSON-format output.
Microsoft’s investment in Mistral AI is significant but tiny compared to its $13 billion stake in OpenAI and Google and Amazon’s investments in Anthropic, which amount to $2 billion and $4 billion respectively.
Mistral AI and Microsoft will collaborate to train bespoke models for customers including European governments.
Behind the news: Mistral AI was founded in early 2023 by engineers from Google and Meta. The French government has touted the company as a home-grown competitor to U.S.-based leaders like OpenAI. France’s representatives in the European Commission argued on Mistral’s behalf to loosen the European Union’s AI Act oversight on powerful AI models. 

Yes, but: Mistral AI’s partnership with Microsoft has divided European lawmakers and regulators. The European Commission, which already was investigating Microsoft’s agreement with OpenAI for potential breaches of antitrust law, plans to investigate the new partnership as well. Members of President Emmanuel Macron’s Renaissance party criticized the deal’s potential to give a U.S. company access to European users’ data. However, other French lawmakers support the relationship.

Why it matters: The partnership between Mistral AI and Microsoft gives the startup crucial processing power for training large models and greater access to potential customers around the world. It gives the tech giant greater access to the European market. And it gives Azure customers access to a high-performance model that’s tailored to Europe’s unique regulatory environment.

We’re thinking: Mistral AI has made impressive progress in a short time, especially relative to the resources at its disposal as a startup. Its partnership with a leading hyperscaler is a sign of the tremendous processing and distribution power that remains concentrated in the large, U.S.-headquartered cloud companies.
"""

In [37]:
summ_prompt = f"""
You are a commentator. Your task is to write a report on a newsletter. 
When presented with the newsletter, come up with interesting questions to ask,
and answer each question. 
Afterward, combine all the information and write a report in the markdown
format. 

# Newsletter: 
{newsletter}

# Instructions: 
## Summarize:
In clear and concise language, summarize the key points and themes 
presented in the newsletter.

## Interesting Questions: 
Generate three distinct and thought-provoking questions that can be 
asked about the content of the newsletter. For each question:
- After "Q: ", describe the problem 
- After "A: ", provide a detailed explanation of the problem addressed 
in the question.- Enclose the ultimate answer in <>.

## Write a analysis report
Using the summary and the answers to the interesting questions, 
create a comprehensive report in Markdown format. 
"""

report = mistral_chat(summ_prompt)
print(report)

# Summary

Mistral AI, a European AI startup, has unveiled two new large language models (LLMs), Mistral Large and Mistral Small, joining its earlier Mistral Medium model. The company has formed a strategic partnership with Microsoft, which includes a $16.3 million investment and an agreement to distribute Mistral Large on Azure. The models are available for free trials and custom deployments, with both models supporting multilingual capabilities (French, German, Spanish, Italian) and function calling.

Mistral Large outperformed competitors like Claude 2, Gemini Pro, and Llama 2 70B on the MMLU benchmark (81.2%) but fell short of GPT-4. Mistral Small, optimized for latency and cost, achieved 72.2% on the same benchmark. The partnership with Microsoft provides Mistral AI with access to Azure’s computing infrastructure and global distribution, while Microsoft gains a foothold in the European AI market.

The alliance has sparked regulatory scrutiny, with the European Commission investiga

In [38]:
import requests

resp = requests.post(
    "http://127.0.0.1:5000/chat",
    json={"message": "My order is late, where is it?"}
)

print(resp.status_code)
print(resp.json())


200
{'intent': 'order_status', 'reply': '**Support Reply:**\nThank you for reaching out! I’d be happy to help track your order. Could you please share your order ID or the email associated with the purchase? This will allow me to check the latest status for you.\n\n*(If the user provides the details, proceed with tracking. If not, ask for one more piece of info, e.g., "Was this order placed online or in-store?")*\n\nIf the issue persists, I can escalate this to a human agent—would you like me to do that? If so, please provide your preferred contact method (email/phone).'}
